### Setup

In [1]:
import ccxt
import pandas as pd

test_key = '4b12f48bb18c3c4f4bab03ad7020bbdc2d2a27c972bbce4e637bfa52c6b679e8'
test_secret = '5814f7cf9fd72320ca56bde829c8c77baad974d0d4258bca03ff4fba392b735e'

symbol = 'ETH/USDT'
symbol_binance = symbol.replace('/', '')

exchange = ccxt.binance({
    'apiKey': test_key,
    'secret': test_secret,
    'options': {
        'defaultType': 'future',  # Esto indica que operarás en Futuros
    },
})

exchange.set_sandbox_mode(True)
exchange.load_markets();

In [2]:
float(exchange.fetch_balance()['info']['availableBalance'])*0.9

13500.911480634

### Place orders

In [3]:
leverage = 5
amount_in_usdt = float(exchange.fetch_balance()['info']['availableBalance'])*0.9
symbol = 'ETH/USDT'

try:
    # Set margin mode to isolated and leverage for the symbol
    margin_mode_response = exchange.set_margin_mode(
        marginMode="ISOLATED",
        symbol=symbol_binance,
        params={'leverage': leverage}  
    )

    # Set leverage for the symbol
    response = exchange.fapiPrivatePostLeverage({
        'symbol': symbol_binance,
        'leverage': leverage
    })
    print(f"Leverage set to {leverage}x for {symbol}")

    # Get the current price
    ticker = exchange.fetch_ticker(symbol)
    current_price = ticker['last']
    print(f"Current price of {symbol}: {current_price} USDT")

    # Calculate the amount to buy
    amount_in_symbol = (amount_in_usdt / current_price) * leverage
    print(f"Buying {amount_in_symbol:.8f} {symbol.split('/')[0]} (~{amount_in_usdt} USDT)")

    # Set stop loss and take profit prices
    sl_price = current_price * 0.95  # 3% stop loss
    tp_price = current_price * 1.1  # 5% take profit

    # Place market buy order
    market_order = exchange.create_order(
        symbol=symbol,
        type='market',
        side='buy',
        amount=amount_in_symbol,
        params={
            'leverage': leverage  
        }
    )
    print("Market buy order placed:", market_order)

    # Place stop-loss order as a market order (futures)
    sl_order = exchange.create_order(
        symbol=symbol,
        type='STOP_MARKET',  # Futures stop-market order
        side='sell',
        amount=amount_in_symbol,
        params={
            'leverage': leverage,  
            'stopPrice': sl_price,  # stopPrice triggers the market order
            'reduceOnly': True,     # Ensures this order only closes the position
            'timeInForce': 'GTE_GTC'  # Keeps the order valid until triggered or canceled
        }
    )
    print("Stop-loss market order placed:", sl_order)

    # Place take-profit order as a market order (futures)
    tp_order = exchange.create_order(
        symbol=symbol,
        type='TAKE_PROFIT_MARKET',  # Futures take-profit market order
        side='sell',
        amount=amount_in_symbol,
        params={
            'leverage': leverage,  
            'stopPrice': tp_price,  # stopPrice triggers the market order
            'reduceOnly': True,     # Ensures this order only closes the position
            'timeInForce': 'GTE_GTC'  # Keeps the order valid until triggered or canceled
        }
    )
    print("Take-profit market order placed:", tp_order)

except ccxt.BaseError as e:
    print(f"An error occurred: {e}")


Leverage set to 5x for ETH/USDT
Current price of ETH/USDT: 3313.32 USDT
Buying 20.37369086 ETH (~13500.911480634 USDT)
Market buy order placed: {'info': {'orderId': '1472199376', 'symbol': 'ETHUSDT', 'status': 'FILLED', 'clientOrderId': 'x-xcKtGhcuf79a64541bb0da986fe104', 'price': '0.00', 'avgPrice': '3348.21707', 'origQty': '20.373', 'executedQty': '20.373', 'cumQty': '20.373', 'cumQuote': '68213.22634', 'timeInForce': 'GTC', 'type': 'MARKET', 'reduceOnly': False, 'closePosition': False, 'side': 'BUY', 'positionSide': 'BOTH', 'stopPrice': '0.00', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': '0', 'updateTime': '1734962500260'}, 'id': '1472199376', 'clientOrderId': 'x-xcKtGhcuf79a64541bb0da986fe104', 'timestamp': 1734962500260, 'datetime': '2024-12-23T14:01:40.260Z', 'lastTradeTimestamp': 1734962500260, 'lastUpdateTimestamp': 1734962500260, 'symbol': 'ETH/USDT:USDT', 'type': 'market

### Check orders

In [4]:
symbol = 'ETH/USDT'
open_orders = exchange.fetch_open_orders(symbol)
closed_orders = exchange.fetch_closed_orders(symbol)
positions = exchange.fetch_positions()

len(open_orders), len(positions)

(2, 1)

In [6]:
closed_orders[-1]

{'info': {'orderId': '4070862668',
  'symbol': 'BTCUSDT',
  'status': 'FILLED',
  'clientOrderId': 'x-xcKtGhcuc4db0cf3938f28e11d29c2',
  'price': '0',
  'avgPrice': '98200.00000',
  'origQty': '0.004',
  'executedQty': '0.004',
  'cumQuote': '392.80000',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'reduceOnly': True,
  'closePosition': False,
  'side': 'SELL',
  'positionSide': 'BOTH',
  'stopPrice': '98378.20',
  'workingType': 'CONTRACT_PRICE',
  'priceMatch': 'NONE',
  'selfTradePreventionMode': 'NONE',
  'goodTillDate': '0',
  'priceProtect': False,
  'origType': 'STOP_MARKET',
  'time': '1733976064279',
  'updateTime': '1733991661952'},
 'id': '4070862668',
 'clientOrderId': 'x-xcKtGhcuc4db0cf3938f28e11d29c2',
 'timestamp': 1733976064279,
 'datetime': '2024-12-12T04:01:04.279Z',
 'lastTradeTimestamp': 1733991661952,
 'lastUpdateTimestamp': 1733991661952,
 'symbol': 'BTC/USDT:USDT',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'reduceOnly': True,
 'side': 'sell'

In [16]:
closed_orders[1]

{'info': {'orderId': '4070647201',
  'symbol': 'BTCUSDT',
  'status': 'FILLED',
  'clientOrderId': 'x-xcKtGhcu9bfa7d7c0273a6c0bec04a',
  'price': '0',
  'avgPrice': '103465.80000',
  'origQty': '0.005',
  'executedQty': '0.005',
  'cumQuote': '517.32900',
  'timeInForce': 'GTC',
  'type': 'MARKET',
  'reduceOnly': True,
  'closePosition': False,
  'side': 'SELL',
  'positionSide': 'BOTH',
  'stopPrice': '104265',
  'workingType': 'CONTRACT_PRICE',
  'priceMatch': 'NONE',
  'selfTradePreventionMode': 'NONE',
  'goodTillDate': '0',
  'priceProtect': False,
  'origType': 'TAKE_PROFIT_MARKET',
  'time': '1733940305364',
  'updateTime': '1733960798084'},
 'id': '4070647201',
 'clientOrderId': 'x-xcKtGhcu9bfa7d7c0273a6c0bec04a',
 'timestamp': 1733940305364,
 'datetime': '2024-12-11T18:05:05.364Z',
 'lastTradeTimestamp': 1733960798084,
 'lastUpdateTimestamp': 1733960798084,
 'symbol': 'BTC/USDT:USDT',
 'type': 'market',
 'timeInForce': 'GTC',
 'postOnly': False,
 'reduceOnly': True,
 'side': 

In [38]:
positions[0]

{'info': {'symbol': 'ETHUSDT',
  'positionSide': 'BOTH',
  'positionAmt': '0.026',
  'entryPrice': '3799.89',
  'breakEvenPrice': '3801.409956',
  'markPrice': '3655.04932721',
  'unRealizedProfit': '-3.76585749',
  'liquidationPrice': '2930.45962582',
  'isolatedMargin': '19.22029199',
  'notional': '95.03128250',
  'marginAsset': 'USDT',
  'isolatedWallet': '22.98614948',
  'initialMargin': '19.00625650',
  'maintMargin': '0.47515641',
  'positionInitialMargin': '19.00625650',
  'openOrderInitialMargin': '0',
  'adl': '0',
  'bidNotional': '0',
  'askNotional': '0',
  'updateTime': '1733862306590'},
 'id': None,
 'symbol': 'ETH/USDT:USDT',
 'contracts': 0.026,
 'contractSize': 1.0,
 'unrealizedPnl': -3.76585749,
 'leverage': None,
 'liquidationPrice': 2930.45962582,
 'collateral': 19.22029199,
 'notional': 95.0312825,
 'markPrice': 3655.04932721,
 'entryPrice': 3799.89,
 'timestamp': 1733862306590,
 'initialMargin': 19.0062565,
 'initialMarginPercentage': 0.2,
 'maintenanceMargin': 0

In [17]:
# Replace with your actual exchange instance and symbol
order_id = '1469497758'
symbol = 'ETH/USDT'

try:
    response = exchange.cancel_order(id=order_id, symbol=symbol)
    print(f"Order {order_id} canceled successfully.")
    print(response)
except Exception as e:
    print(f"Failed to cancel order {order_id}: {e}")


Order 1469497758 canceled successfully.
{'info': {'orderId': '1469497758', 'symbol': 'ETHUSDT', 'status': 'CANCELED', 'clientOrderId': 'x-xcKtGhcued082429ace6d545a9277e', 'price': '0.00', 'avgPrice': '0.00', 'origQty': '0.012', 'executedQty': '0.000', 'cumQty': '0.000', 'cumQuote': '0.00000', 'timeInForce': 'GTE_GTC', 'type': 'STOP_MARKET', 'reduceOnly': True, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '3928.31', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'STOP_MARKET', 'priceMatch': 'NONE', 'selfTradePreventionMode': 'NONE', 'goodTillDate': '0', 'updateTime': '1733775130508'}, 'id': '1469497758', 'clientOrderId': 'x-xcKtGhcued082429ace6d545a9277e', 'timestamp': 1733775130508, 'datetime': '2024-12-09T20:12:10.508Z', 'lastTradeTimestamp': None, 'lastUpdateTimestamp': 1733775130508, 'symbol': 'ETH/USDT:USDT', 'type': 'stop_market', 'timeInForce': 'GTE_GTC', 'postOnly': False, 'reduceOnly': True, 'side': 'sell', 'price': None, 'tr

In [ ]:
from datetime import datetime, timedelta
import logging
import requests

logger = logging.getLogger(__name__)


class BinanceFuturesTrader:
    def __init__(self, api_key, api_secret, sandbox=True):
        self.api_url = "http://10.0.137.230:6378/enqueue"
        self.exchange = ccxt.binance({
            'apiKey': api_key,
            'secret': api_secret,
            'options': {'defaultType': 'future'},
        })
        self.exchange.set_sandbox_mode(sandbox)
        self.exchange.load_markets()
        self.sl_order = {}
        self.tp_order = {}

        # Load spot market
        self.spot_exchange = ccxt.binance({
            'options': {'defaultType': 'spot'},
        })
        self.spot_exchange.load_markets()
    

    def get_dataframe(self, symbol: str, n_candles: int) -> pd.DataFrame:
        ohlcv = self.spot_exchange.fetch_ohlcv(symbol, '1h', limit=n_candles)
        cols = ['date', 'open', 'high', 'low', 'close', 'volume']
        df = pd.DataFrame(ohlcv, columns=cols)
        df['date'] = pd.to_datetime(df['date'], unit='ms')
        df['date'] = df['date'].dt.tz_localize('UTC')

        return df


    def call_api(self, symbol, n_candles) -> list:
        # Get spot data
        dataframe = self.get_dataframe(symbol, n_candles)
        
        # Prepare JSON 
        data_json = {
            "pair": symbol.replace('/USDT', ''),
            "data": {
                "open": dataframe['open'].tolist(),
                "high": dataframe['high'].tolist(),
                "low": dataframe['low'].tolist(),
                "close": dataframe['close'].tolist(),
                "volume": dataframe['volume'].tolist(),
                "date": dataframe['date'].apply(lambda x: x.isoformat() if isinstance(x, pd.Timestamp) else x).tolist() 
                }
        }

        # solicitud POST a la API
        try:
            response = requests.post(self.api_url, json=data_json)
            response.raise_for_status()  # Comprobar si hay errores en la solicitud
            predictions = response.json().get('prediction', [])

            logger.info('REQUEST PREDICTION')
            logger.info(str(datetime.now()))
            logger.info(str(predictions))

            return predictions
        
        except requests.exceptions.RequestException as e:
            print(f"Error al llamar a la API: {e}")
            return []
    

    def set_leverage(self, symbol, leverage):
        try:
            symbol_binance = symbol.replace('/', '')
            response = self.exchange.fapiPrivatePostLeverage({
                'symbol': symbol_binance,
                'leverage': leverage
            })
            print(f"Leverage set to {leverage}x for {symbol}")
        except ccxt.BaseError as e:
            print(f"Failed to set leverage for {symbol}: {e}")


    def place_orders(self, symbol, amount_in_usdt, leverage=5, sl_pct=0.97, tp_pct=1.05):
        try:
            self.set_leverage(symbol, leverage)

            # Fetch current price
            ticker = self.exchange.fetch_ticker(symbol)
            current_price = ticker['last']

            # Calculate the amount to buy
            amount_in_symbol = amount_in_usdt / current_price
            print(f"Buying {amount_in_symbol:.8f} {symbol.split('/')[0]} (~{amount_in_usdt} USDT)")

            # Set SL and TP prices
            sl_price = current_price * sl_pct
            tp_price = current_price * tp_pct

            # Place market buy order
            market_order = self.exchange.create_order(
                symbol=symbol,
                type='market',
                side='buy',
                amount=amount_in_symbol
            )
            print("Market buy order placed:", market_order)

            # Place stop-loss order
            sl_order = self.exchange.create_order(
                symbol=symbol,
                type='STOP_MARKET',
                side='sell',
                amount=amount_in_symbol,
                price=sl_price,
                params={'stopPrice': sl_price,  # stopPrice triggers the market order
                        'reduceOnly': True,     # Ensures this order only closes the position
                        'timeInForce': 'GTE_GTC'  # Keeps the order valid until triggered or canceled
                        }
            )
            print("Stop-loss order placed:", sl_order)

            # Place take-profit order
            tp_order = self.exchange.create_order(
                symbol=symbol,
                type='TAKE_PROFIT_MARKET',
                side='sell',
                amount=amount_in_symbol,
                price=tp_price,
                params={'stopPrice': tp_price,  # takeProfit triggers the market order
                        'reduceOnly': True,     # Ensures this order only closes the position
                        'timeInForce': 'GTE_GTC'  # Keeps the order valid until triggered or canceled
                        }
            )
            print("Take-profit order placed:", tp_order)

        except ccxt.BaseError as e:
            print(f"An error occurred while placing orders: {e}")

    def update_orders(self, symbol):
        open_orders = self.exchange.fetch_open_orders(symbol)
        
        self.sl_order[symbol] = None
        self.tp_order[symbol] = None
        
        # Assign orders if found
        for order in open_orders:
            if order['info']['origType'] == 'TAKE_PROFIT_MARKET':
                self.sl_order[symbol] = order
            elif order['info']['origType'] == 'STOP_MARKET':
                self.tp_order[symbol] = order
        
        sl_order = self.sl_order[symbol]
        tp_order = self.tp_order[symbol]
        
        # Case 1: Both SL and TP are open
        if sl_order and tp_order:
            return False  # No need to trade
        
        # Case 2: No open orders
        if not sl_order and not tp_order:
            return True  # Ready to trade
        
        # Case 3: One order is left open, cancel the stray order
        try:
            if not sl_order:
                self.exchange.cancel_order(tp_order['id'], symbol)
            elif not tp_order:
                self.exchange.cancel_order(sl_order['id'], symbol)
            return True  # After cancellation, ready to trade
        except ccxt.BaseError as e:
            print(f"Error while closing orders for {symbol}: {e}")
            return False  # Do not trade if error occurs
        
    def check_cooldown(self, symbol, n_hours):
        try:
            closed_orders = self.exchange.fetch_closed_orders(symbol)
            cutoff_time = datetime.now() - timedelta(hours=n_hours)

            # Filter closed orders within last n_hours
            recent_orders = [
                order for order in closed_orders 
                if datetime.fromtimestamp(order['lastTradeTimestamp'] / 1000) >= cutoff_time
            ]

            # Sort by timestamp in descending order
            recent_orders.sort(key=lambda x: x['lastTradeTimestamp'], reverse=True) 
            return len(recent_orders) > 0 and recent_orders[0]['info']['origType'] == 'STOP_MARKET'
        
        except Exception as e:
            print(f"Error in check_cooldown for {symbol}: {e}")
            return False

    def check_entry_signal(self, symbol):
        cooldown = 24
        n_candles = 50
        leverage = 5
        SL = 0.97
        TP = 1.05
        usdt_amount = 10

        # Check and update order status, skip trading if position is open
        is_ready_to_trade = self.update_orders(symbol)
        if not is_ready_to_trade:
            print(f"Skipping trade for {symbol} because of open position.")
            return
        
        # Check for cooldown
        if self.check_cooldown(symbol, cooldown):
            print(f"Skipping trade for {symbol} because of recent loss.")
            return 
        
        # Compute prediction
        predictions = self.call_api(symbol=symbol, n_candles=n_candles)

        if len(predictions) > 0 and predictions[-1]:
            print(f"Entry signal detected for {symbol}. Placing orders...")
            self.place_orders(symbol, usdt_amount, leverage=leverage, sl_pct=SL, tp_pct=TP)
        else:
            print(f"No entry signal detected for {symbol}.")